# Overview

In this notebook we'll work with and explore a dataset of used cars from eBay kleinanzeign, a classifieds section of the German eBay website. 

This dataset was originall uploaded to [Kaggle](https://www.kaggle.com/orgesleka/used-cars-database/data), but we'll work with a sample of 50,000 data points from the full dataset in this notebook so that it's easy for anyone reading this to replicate with ease.

The aim of this short project is to clean and analyze the used car listings.

The dataset was originally in German, but has been converted to English for our use here. The features are as follows:

- ```dateCrawled``` - When this ad was first crawled. All field-values are taken from this date.

- ```name``` - Name of the car.

- ```seller``` - Whether the seller is private or a dealer.

- ```offerType``` - The type of listing

- ```price``` - The price on the ad to sell the car.

- ```abtest``` - Whether the listing is included in an A/B test.

- ```vehicleType``` - The vehicle Type.
- ```yearOfRegistratio```n - The year in which the car was first registered.
- ```gearbox``` - The transmission type.
- ```powerPS``` - The power of the car in PS.
- ```model``` - The car model name.
- ```kilometer``` - How many kilometers the car has driven.
- ```monthOfRegistration``` - The month in which the car was first registered.
- ```fuelType``` - What type of fuel the car uses.
- ```brand``` - The brand of the car.
- ```notRepairedDamage``` - If the car has a damage which is not yet repaired.
- ```dateCreated``` - The date on which the eBay listing was created.
- ```nrOfPictures``` - The number of pictures in the ad.
- ```postalCode``` - The postal code for the location of the vehicle.
- ```lastSeenOnline``` - When the crawler saw this ad last online.


In [3]:
import pandas as pd
import numpy as np
import re

In [4]:
autos = pd.read_csv("autos.csv", encoding='Latin-1')

In [5]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [6]:
print("There are {} features in our dataset and {} rows.".format(len(autos.columns), len(autos)))

There are 20 features in our dataset and 50000 rows.


In [7]:
autos.describe()

,yearOfRegistration,powerPS,monthOfRegistration,nrOfPictures,postalCode
count,50000.000000,50000.000000,50000.000000,50000.0,50000.000000
mean,2005.073280,116.355920,5.723360,0.0,50813.627300
std,105.712813,209.216627,3.711984,0.0,25779.747957
min,1000.000000,0.000000,0.000000,0.0,1067.000000
25%,1999.000000,70.000000,3.000000,0.0,30451.000000
50%,2003.000000,105.000000,6.000000,0.0,49577.000000
75%,2008.000000,150.000000,9.000000,0.0,71540.000000
max,9999.000000,17700.000000,12.000000,0.0,99998.000000


In [8]:
np.sum(autos.isnull())

dateCrawled               0
name                      0
seller                    0
offerType                 0
price                     0
abtest                    0
vehicleType            5095
yearOfRegistration        0
gearbox                2680
powerPS                   0
model                  2758
odometer                  0
monthOfRegistration       0
fuelType               4482
brand                     0
notRepairedDamage      9829
dateCreated               0
nrOfPictures              0
postalCode                0
lastSeen                  0
dtype: int64

## Preliminary thoughts on data
Right off the bat we can see that their are 20 features but only 5 of them have numerical data. I'll note a few of the features that will require cleaning because they represent 3 of the common techniques we'll use while cleaning the data. We'll have to look in more detail into ```nrOfPictures``` feature because with a max value of 0.0 and a min value of 0.0, it is a great candidate to be dropped to reduce the complexity of any future models we might want to build with this data. Other data features like ```vehicleType``` and ```gearbox``` are great candidates for one-hot encoding because there are only a handful of values each feature. Lastly, we can strip 'minute' values within features with dates like ```dateCreated``` so that it's easier to work with.

In [59]:
col_names = list(autos.columns)

In [60]:
col_names

['dateCrawled',
 'name',
 'seller',
 'offerType',
 'price',
 'abtest',
 'vehicleType',
 'yearOfRegistration',
 'gearbox',
 'powerPS',
 'model',
 'odometer',
 'monthOfRegistration',
 'fuelType',
 'brand',
 'notRepairedDamage',
 'dateCreated',
 'nrOfPictures',
 'postalCode',
 'lastSeen']

Taking a quick look at the columns above, we can see that although the column names are understandable, we can make them a little clearer to help any data scientist or engineer downstream work with the data.

In order to make the column names more descriptive, we'll first create a dictionary of 'old' to 'new' values we know we want to change the wording on. Then because pythonistas perfer snake case over camel case, we can do a bit of regex magic to convert our column names.

In [61]:
new_cols = col_names.copy()

In [62]:
dictionary = {
    "yearOfRegistration":'registration_year',
    "monthOfRegistration":"registration_month",
    "notRepairedDamage":"unrepaired_damage",
    "dateCreated":'ad_created'
} 

In [63]:
for ind, item in enumerate(new_cols):
    new_cols[ind] = dictionary.get(item, item)
    
for ind, name in enumerate(new_cols):
    new_cols[ind] = re.sub(r'(?<!^)(?=[A-Z])', '_', name).lower()

In [64]:
new_cols

['date_crawled',
 'name',
 'seller',
 'offer_type',
 'price',
 'abtest',
 'vehicle_type',
 'registration_year',
 'gearbox',
 'power_p_s',
 'model',
 'odometer',
 'registration_month',
 'fuel_type',
 'brand',
 'unrepaired_damage',
 'ad_created',
 'nr_of_pictures',
 'postal_code',
 'last_seen']

In [65]:
autos.columns = new_cols

In [66]:
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_p_s,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50
